In [1]:
import numpy as np
import pandas as pd
import os, random
from fastai.vision.all import *
import shutil
import pydicom
import cv2
import glob

from tqdm.notebook import tqdm
from joblib import Parallel, delayed

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
new_learn = load_learner('/kaggle/input/atd-baseline1-model/model.pt', cpu = False)

In [3]:
TEST_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images/'
SAVE_FOLDER = '/kaggle/working/test_png/'
SIZE = 512
EXTENSION = 'png'

os.makedirs(SAVE_FOLDER, exist_ok = True)
#os.makedirs(TEST_PATH, exist_ok = True)
#!cp -r /kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/10004 /kaggle/working/test_images/
#!cp -r /kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/10005 /kaggle/working/test_images/
print('Number of test patients:', len(os.listdir(TEST_PATH)))

Number of test patients: 3


In [4]:
def process(patient, size = SIZE, save_folder = "", data_path = ""):
    
    num_scans = 10
    for study in sorted(os.listdir(data_path + patient)):
        imgs = {}
        for f in sorted(random.choices(glob.glob(data_path + f"{patient}/{study}/*.dcm"), k = num_scans)):
            
            dicom = pydicom.dcmread(f)
            pos_z = dicom[(0x20, 0x32)].value[-1]
            
            img = dicom.pixel_array
            img = (img - img.min())/(img.max() - img.min() + 1e-6)
            
            imgs[pos_z] = img
        
        for i, k in enumerate(sorted(imgs.keys())):
            
            img = imgs[k]
            
            img = cv2.resize(img, (size, size))
            cv2.imwrite(save_folder + f"{patient}_{study}_{i}.png", (img * 255).astype(np.uint8))

In [5]:
import multiprocessing
print('Number of cores: ', multiprocessing.cpu_count())

Number of cores:  2


In [6]:
patients = os.listdir(TEST_PATH)

_ = Parallel(n_jobs = 2)(
    delayed(process)(patient, size=512, save_folder=SAVE_FOLDER, data_path=TEST_PATH)
    for patient in tqdm(patients)
)
    
files = get_image_files(SAVE_FOLDER)

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
test_dl = new_learn.dls.test_dl(files, with_labels = False)
test_dl.batch_size = 256

preds = new_learn.get_preds(dl = test_dl)[0].numpy()

In [8]:
test_files_probs = pd.DataFrame()

test_files_probs['fname'] = pd.Series(os.listdir(SAVE_FOLDER), dtype = 'string')

test_files_probs['bowel_healthy'] = pd.Series(1 - preds[:, 0])
test_files_probs['bowel_injury'] = pd.Series(preds[:, 0])
test_files_probs['extravasation_healthy'] = pd.Series(1 - preds[:, 1])
test_files_probs['extravasation_injury'] = pd.Series(preds[:, 1])
test_files_probs['extravasation_healthy'] = pd.Series(1 - preds[:, 1])
test_files_probs['kidney_healthy'] = pd.Series(1 - preds[:, 2])
test_files_probs['kidney_low'] = pd.Series(preds[:, 2]/2)
test_files_probs['kidney_high'] = pd.Series(preds[:, 2]/2)
test_files_probs['liver_healthy'] = pd.Series(1 - preds[:, 3])
test_files_probs['liver_low'] = pd.Series(preds[:, 3]/2)
test_files_probs['liver_high'] = pd.Series(preds[:, 3]/2)
test_files_probs['spleen_healthy'] = pd.Series(1 - preds[:, 4])
test_files_probs['spleen_low'] = pd.Series(preds[:, 4]/2)
test_files_probs['spleen_high'] = pd.Series(preds[:, 4]/2)

#test_files_probs

In [9]:
patient_id_list = []
for idx, fname in enumerate(test_files_probs['fname']):
    patient_id_list.append(test_files_probs.fname[idx].split('_')[0])
    
test_files_probs['patient_id'] = pd.Series(patient_id_list, dtype = 'string')
#test_files_probs

In [10]:
x = test_files_probs.groupby('patient_id').mean()

x.insert(0, 'patient_id', x.index)
x.reset_index(drop = True)

/tmp/ipykernel_24/935710440.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = test_files_probs.groupby('patient_id').mean()


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.598430,0.401570,0.214198,0.785802,0.640973,0.179513,0.179513,0.681621,0.159190,0.159190,0.591546,0.204227,0.204227
1,50046,0.559621,0.440379,0.220175,0.779825,0.862120,0.068940,0.068940,0.768038,0.115981,0.115981,0.295738,0.352131,0.352131
2,63706,0.727782,0.272218,0.641310,0.358690,0.946498,0.026751,0.026751,0.385548,0.307226,0.307226,0.465114,0.267443,0.267443


In [11]:
x.to_csv('submission.csv', header = True, index = False)